# $\color{purple}{\text{Understanding Missing Data and How to Deal with It (Part 5)}}$

## $\color{purple}{\text{Advanced Imputation Techniques}}$

### $\color{purple}{\text{Colab Environmental Setup}}$

### $\color{purple}{\text{Libraries for this lesson}}$

In [1]:
import pandas as pd
import numpy as np
from helpers import stat_comparison, spotlight_donors, ImputationDisplayer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from autoimpute.imputations import SingleImputer
from autoimpute.imputations import MultipleImputer
from autoimpute.imputations import MiceImputer

In [113]:
df = pd.read_csv('data/full_set.csv')
mar_df = pd.read_csv('data/mar_set.csv')
displayer = ImputationDisplayer(mar_df)

## $\color{purple}{\text{Multivariate Imputation}}$
Conventional Multivariate Imputation falls into 2 categories
* Regression Imputation
* Hot Deck Imputation

Another cutting edge method worth mentioning
* Neural Network Autoencoder

## $\color{purple}{\text{Regression Imputation}}$

General Technique:
Use Regression/Classification Models to impute Numeric/Categorical Missing Values
* Linear Regression
* Stocastic Linear Regression
* Logistic Regression
* Other Possibilities (generally unexplored)
  * Random Forest
  * Decision Trees
  * KNN

### $\color{purple}{\text{Linear Regression}}$

* Works with MAR
* Can impute illegal (out of bounds) values
* Can under estimate variance/covariance

In [3]:
linear_regressor = LinearRegression()

In [4]:
rest = ['feature b', 'feature c', 'feature d', 'uncorrelated']
full_data = mar_df.dropna()
linear_regressor.fit(full_data[rest], full_data['feature a'])
predicted = linear_regressor.predict(mar_df[rest])

In [5]:
imputed=mar_df.assign(**{'feature a': mar_df['feature a'].where(~mar_df['feature a'].isnull(), predicted)})

In [6]:
stat_comparison(df, imputed, 'feature a')

,Original,With Missing Data,difference,percentage
mean,2.367217,2.367493,0.000276,0.011674
median,2.380412,2.384461,0.004049,0.170097
stdev,1.280482,1.278774,0.001708,0.133419


In [8]:
displayer(imputed, 15)

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,2.884180,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471


### $\color{purple}{\text{Stochastic Regression}}$
* Extends Linear Regression by adding noise modelling the residuals
* Better simulates variance
* Can also produce out of bounds values

In [10]:
residual = mar_df['feature a'] - predicted
residual.mean()
residual.std()

0.15480550226374812

In [12]:
residual_noise=np.random.normal(residual.mean(), residual.std(), 20000)
predicted+=residual_noise

In [13]:
imputed=mar_df.assign(**{'feature a': mar_df['feature a'].where(~mar_df['feature a'].isnull(), predicted)})

In [14]:
stat_comparison(df, imputed, 'feature a')

,Original,With Missing Data,difference,percentage
mean,2.367217,2.367127,0.000089,0.003775
median,2.380412,2.380770,0.000358,0.015051
stdev,1.280482,1.280066,0.000416,0.032457


In [9]:
displayer(imputed, 15)

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,2.884180,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471


In [15]:
imputer=SingleImputer('least squares')
imputations = imputer.fit_transform(mar_df)

In [52]:
from autoimpute.imputations import SingleImputer
imputer=SingleImputer('stochastic')
imputations = imputer.fit_transform(mar_df)

## $\color{purple}{\text{Hot Deck Imputation}}$
* General Idea is to randomly sample imputed values from remaining good values.
* Doesn't impute out of bounds values

How it works:
* For each missing value, a set of donors is selected from good values
* A value is randomly selected from the set of donors
* Donors are selected based on some metric based algorithm

The `demo_mar.csv` dataset is the first 10 entries from one of my earlier runs. It has one missing value in `feature a`

In [18]:
demo_df = pd.read_csv('data/demo_mar.csv')
demo_df

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,NaN,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471


We use Euclidean distance to demonstrate how Hot Deck Imputation works, but in practice the metric is usually more statistically based and complex. For simplicity we add a `distance` feature.

In [19]:
def distance(x):
    return np.linalg.norm((x-demo_df.iloc[7]).dropna())
    
demo_df['distance'] = demo_df.apply(distance, axis=1)

In [20]:
demo_df

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,NaN,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085


[Van Buuren](https://stefvanbuuren.name/fimd/) identifies 4 methods of selecting donors

#### Method 1: (Single Donor)

Pick the sample closest to the missing value

In [21]:
donor = demo_df.dropna().nsmallest(1, 'distance')
spotlight_donors(demo_df,donor)

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,nan,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085


#### Method 2:

Donors selected from all points under a fixed threshold

In [23]:
threshold = 2
donors = demo_df.dropna()[demo_df.dropna().distance<threshold]['feature a']
spotlight_donors(demo_df, donors)

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,nan,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085


#### Method 3:

Closest N points selected as the set of donors

In [24]:
N=3
donors = demo_df.nsmallest(N+1, 'distance').tail(N)['feature a']
spotlight_donors(demo_df, donors)

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,nan,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085


#### Method 4:

Donors are all points, but donor selected randomly based on the distance, closest having higher probability

In [26]:
import random
# Pick with probability inversely proportionally to distance
weights = 1/demo_df.dropna()['distance']
random.choices(demo_df.dropna()['feature a'].to_list(), k=1, weights=weights.to_list())

[1.5175094484003897]

## $\color{purple}{\text{Predictive Mean Matching}}$
Uses linear interpolation as part of the metric.

Basically, the donors are selected from those observations whose predicted values from linear regression most closely matches that predicted from the missing value.


In [27]:
from sklearn.linear_model import LinearRegression
linear_regressor=LinearRegression()

In [28]:
demo_df = pd.read_csv('data/demo_mar.csv')

In [29]:
rest = ['feature b', 'feature c', 'feature d', 'uncorrelated']
full_data = demo_df.dropna()
linear_regressor.fit(full_data[rest], full_data['feature a'])
demo_df['regression'] = linear_regressor.predict(demo_df[rest])

In [30]:
demo_df['distance']=np.abs(demo_df.regression-demo_df.regression.iloc[7])

In [31]:
N=3
donors = demo_df.dropna().sort_values('distance').iloc[0:N]['feature a']
spotlight_donors(demo_df, donors, 7)

,feature a,feature b,feature c,feature d,uncorrelated,regression,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,1.520445,1.320369
1,2.536323,4.295391,2.104137,1.348000,0.998701,2.542722,0.298092
2,4.043034,5.872276,3.559629,3.274061,0.403823,4.117501,1.276687
3,0.082752,3.761743,-0.440590,1.031832,0.281023,0.030163,2.810651
4,0.196684,3.793343,1.016462,-0.667764,0.165431,0.202129,2.638685
5,2.560068,4.446726,2.420763,0.973363,0.166179,2.363152,0.477662
6,4.027199,5.079975,4.582185,0.876607,0.420479,4.023186,1.182373
7,nan,5.339294,3.138633,1.611132,0.229141,2.840814,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,2.801720,0.039094
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,-0.073960,2.914774


Predictive Mean Matching is the preferred imputation method, but can be computationally expensive

In [11]:
from autoimpute.imputations import SingleImputer
demo_df = mar_df[0:100].copy()
imputer=SingleImputer('pmm')
imputations = imputer.fit_transform(demo_df)


/opt/conda/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ, beta, alpha]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [12]:
displayer(imputations, 10)

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,3.121393,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471



## $\color{purple}{\text{Categorical Variables}}$

Imputation of categorical variables employs classification in place of regression. Most common is multinomial logistic regression.

In [13]:
cat_mar_df = pd.read_csv('data/categorical_mar.csv')

In [21]:
# A little EDA
cat_mar_df.isnull().sum()

feature a         0
feature b         0
feature c         0
cat feature    4108
dtype: int64

In [17]:
from sklearn.linear_model import LogisticRegression
rest = ['feature a', 'feature b', 'feature c']
from sklearn.preprocessing import LabelEncoder
df = cat_mar_df.dropna()
lr = LogisticRegression(random_state=0, max_iter=1000).fit(df[rest], df['cat feature'])

In [18]:
impute = lr.predict(cat_mar_df[rest])

In [19]:
imputed=cat_mar_df.assign(**{'cat feature': cat_mar_df['cat feature'].where(~cat_mar_df['cat feature'].isnull(), impute)})

## $\color{purple}{\text{Advanced Imputation Techniques: multivariate imputation by chained equations (MICE)}}$
* Often considered the gold standard of imputation
* Is actually more of an imputation blueprint
* Applicable with missingness in multiple columns

In [23]:
dmar_df = pd.read_csv('data/double_mar_set.csv')
missing_df=pd.DataFrame({'feature a': dmar_df['feature a'].isnull(),
                         'feature b': dmar_df['feature b'].isnull()})
displayer = ImputationDisplayer(dmar_df)

#### First step: Impute each missing value with some form of univariate imputation (usually mean or median)

In [109]:
step1_df=dmar_df.fillna({'feature a': dmar_df['feature a'].mean(), 'feature b': dmar_df['feature b'].mean()})
displayer(step1_df, 10)

,feature a,feature b,feature c,feature d,uncorrelated
0,-2.827918,-3.926024,-0.171649,1.599414,0.808661
1,0.827494,-2.458078,2.251578,3.886670,0.194692
2,0.194008,-2.577198,2.882425,3.887535,0.886731
3,-1.907789,-3.792876,0.256099,2.430528,0.493951
4,-0.937901,-3.825973,1.862588,1.099501,0.394353
5,-0.345672,-2.458078,2.023978,3.730747,0.679692
6,0.194008,-3.197703,3.108877,2.058049,0.807557
7,1.563736,-2.458078,3.395702,4.316490,0.489266
8,-0.173297,-2.581213,2.242131,2.828541,0.914358
9,0.365566,-1.798881,2.300123,4.604163,0.908147


#### Second Step: For each column impute using a regression or hot deck technique
Start with `feature a` then `feature b`

##### Clear the missing values for the imputer then impute `feature a`

In [112]:
imputer=SingleImputer('least squares')
step2a_df=imputer.fit_transform(step1_df.assign(**{'feature a': step1_df['feature a'].where(~missing_df['feature a'], np.nan)}))
displayer(step2a_df, 10)

,feature a,feature b,feature c,feature d,uncorrelated
0,-2.827918,-3.926024,-0.171649,1.599414,0.808661
1,0.827494,-2.458078,2.251578,3.886670,0.194692
2,0.899310,-2.577198,2.882425,3.887535,0.886731
3,-1.907789,-3.792876,0.256099,2.430528,0.493951
4,-0.937901,-3.825973,1.862588,1.099501,0.394353
5,-0.345672,-2.458078,2.023978,3.730747,0.679692
6,0.528221,-3.197703,3.108877,2.058049,0.807557
7,1.563736,-2.458078,3.395702,4.316490,0.489266
8,-0.173297,-2.581213,2.242131,2.828541,0.914358
9,0.365566,-1.798881,2.300123,4.604163,0.908147


In [74]:
df=step1_df.assign(**{'feature a': step1_df['feature a'].where(~missing_df['feature a'], np.nan)})
stat_comparison(dmar_df, df, 'feature a')

,Original,With Missing Data,difference,percentage
mean,0.194008,0.194008,0.0,0.0
median,0.131259,0.131259,0.0,0.0
stdev,1.262807,1.262807,0.0,0.0


##### Now impute `feature b`

In [64]:
imputer=SingleImputer('least squares')
step2_df=imputer.fit_transform(step2a_df.assign(**{'feature b': step2a_df['feature b'].where(~missing_df['feature b'], np.nan)}))
displayer(step2_df, 10)

,feature a,feature b,feature c,feature d,uncorrelated
0,-2.827918,-3.926024,-0.171649,1.599414,0.808661
1,0.827494,-2.853203,2.251578,3.886670,0.194692
2,0.894229,-2.577198,2.882425,3.887535,0.886731
3,-1.907789,-3.792876,0.256099,2.430528,0.493951
4,-0.937901,-3.825973,1.862588,1.099501,0.394353
5,-0.345672,-2.013185,2.023978,3.730747,0.679692
6,0.526699,-3.197703,3.108877,2.058049,0.807557
7,1.563736,-2.017757,3.395702,4.316490,0.489266
8,-0.173297,-2.581213,2.242131,2.828541,0.914358
9,0.365566,-1.798881,2.300123,4.604163,0.908147


In [54]:
imputer=SingleImputer('least squares')
step3a_df=imputer.fit_transform(step2_df.assign(**{'feature a': step2_df['feature a'].where(~missing_df['feature a'], np.nan)}))
step3_df=imputer.fit_transform(step3a_df.assign(**{'feature b': step3a_df['feature b'].where(~missing_df['feature b'], np.nan)}))
displayer(step3_df, 10)

,feature a,feature b,feature c,feature d,uncorrelated
0,-2.827918,-3.926024,-0.171649,1.599414,0.808661
1,0.827494,-2.840534,2.251578,3.886670,0.194692
2,1.125480,-2.577198,2.882425,3.887535,0.886731
3,-1.907789,-3.792876,0.256099,2.430528,0.493951
4,-0.937901,-3.825973,1.862588,1.099501,0.394353
5,-0.345672,-2.000889,2.023978,3.730747,0.679692
6,0.585221,-3.197703,3.108877,2.058049,0.807557
7,1.563736,-1.992358,3.395702,4.316490,0.489266
8,-0.173297,-2.581213,2.242131,2.828541,0.914358
9,0.365566,-1.798881,2.300123,4.604163,0.908147


In [115]:
stat_comparison(df, dmar_df, 'feature a')

,Original,With Missing Data,difference,percentage
mean,2.367217,0.194008,2.173208,91.804372
median,2.380412,0.131259,2.249154,94.485886
stdev,1.280482,1.262807,0.017675,1.380349


In [150]:
imputer=MiceImputer(n=1,strategy='least squares')

In [151]:
[each for each in imputer.fit_transform(dmar_df)]

[(1,
         feature a  feature b  feature c  feature d  uncorrelated
  0      -2.827918  -3.926024  -0.171649   1.599414      0.808661
  1       0.827494  -2.804147   2.251578   3.886670      0.194692
  2       1.132261  -2.577198   2.882425   3.887535      0.886731
  3      -1.907789  -3.792876   0.256099   2.430528      0.493951
  4      -0.937901  -3.825973   1.862588   1.099501      0.394353
  ...          ...        ...        ...        ...           ...
  19995  -1.316783  -1.853379   0.632680   4.479027      0.334140
  19996   1.547273  -2.070106   4.070505   3.239217      0.125296
  19997  -0.552074  -2.135666   2.101401   3.162508      0.090778
  19998  -0.089824  -2.312533   2.102443   3.203033      0.139082
  19999   1.240722  -2.072572   3.435945   3.754910      0.778034
  
  [20000 rows x 5 columns])]

## $\color{purple}{\text{Advanced Imputation Techniques: Multiple Imputation}}$

Hot Desk Imputation

Regression Imputation

Multiple Imputation

MICE

In [132]:
from autoimpute.imputations import MultipleImputer
imputer=MultipleImputer(strategy='stochastic')
imputations = imputer.fit_transform(mar_df)

In [133]:
lists=list(imputations)

In [137]:
lists[0][1].head(10)

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,2.953229,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471


In [138]:
lists[1][1].head(10)

,feature a,feature b,feature c,feature d,uncorrelated
0,1.517509,4.229258,2.052726,0.153278,0.014975
1,2.536323,4.295391,2.104137,1.348000,0.998701
2,4.043034,5.872276,3.559629,3.274061,0.403823
3,0.082752,3.761743,-0.440590,1.031832,0.281023
4,0.196684,3.793343,1.016462,-0.667764,0.165431
5,2.560068,4.446726,2.420763,0.973363,0.166179
6,4.027199,5.079975,4.582185,0.876607,0.420479
7,2.976430,5.339294,3.138633,1.611132,0.229141
8,2.743726,4.506330,2.620240,1.362915,0.011719
9,-0.180238,3.148906,0.280848,-0.741796,0.104471


In [141]:
[each[1].iloc[7]['feature a'] for each in lists]

[2.9532294619521022,
 2.9764296840936324,
 2.852605077672565,
 2.772903467827853,
 2.8978580843592163]

In [142]:
[each[1].iloc[6]['feature a'] for each in lists]

[4.027199122849932,
 4.027199122849932,
 4.027199122849932,
 4.027199122849932,
 4.027199122849932]

In [95]:


df = pd.DataFrame({'key': list('ABCD'), 'value': range(4)})




df.style.apply(lambda x: ['background: lightgreen' 
                                  if (x.name == 0 or x.name == 2)
                                  else '' for i in x], axis=1)

,key,value
0,A,0
1,B,1
2,C,2
3,D,3


In [97]:
import pandas as pd
import numpy as np

def red(val):
    color = 'red'
    return 'background-color: %s' % color

def green(val):
    color = 'green'
    return 'background-color: %s' % color

raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
            'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
            'deaths': [523, 52, 25, 616, 43, 234, 523, 62, 62, 73, 37, 35],
            'battles': [5, 42, 2, 2, 4, 7, 8, 3, 4, 7, 8, 9],
            'size': [1045, 957, 1099, 1400, 1592, 1006, 987, 849, 973, 1005, 1099, 1523],
            'veterans': [1, 5, 62, 26, 73, 37, 949, 48, 48, 435, 63, 345],
            'readiness': [1, 2, 3, 3, 2, 1, 2, 3, 2, 1, 2, 3],
            'armored': [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
            'deserters': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
            'origin': ['Arizona', 'California', 'Texas', 'Florida', 'Maine', 'Iowa', 'Alaska', 'Washington', 'Oregon', 'Wyoming', 'Louisana', 'Georgia']}

df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'deaths', 'battles', 'size', 'veterans', 'readiness', 'armored', 'deserters', 'origin'])

df = df.set_index('origin')
print (df)

df.style.applymap(green, subset=pd.IndexSlice['Arizona':'Texas', 'company': 'size']) \
        .applymap(red, subset=pd.IndexSlice['Florida':'Maine', 'veterans': 'armored'])

              regiment company  deaths  battles  size  veterans  readiness  \
origin                                                                       
Arizona     Nighthawks     1st     523        5  1045         1          1   
California  Nighthawks     1st      52       42   957         5          2   
Texas       Nighthawks     2nd      25        2  1099        62          3   
Florida     Nighthawks     2nd     616        2  1400        26          3   
Maine         Dragoons     1st      43        4  1592        73          2   
Iowa          Dragoons     1st     234        7  1006        37          1   
Alaska        Dragoons     2nd     523        8   987       949          2   
Washington    Dragoons     2nd      62        3   849        48          3   
Oregon          Scouts     1st      62        4   973        48          2   
Wyoming         Scouts     1st      73        7  1005       435          1   
Louisana        Scouts     2nd      37        8  1099        63 

,regiment,company,deaths,battles,size,veterans,readiness,armored,deserters
origin,,,,,,,,,
Arizona,Nighthawks,1st,523,5,1045,1,1,1,4
California,Nighthawks,1st,52,42,957,5,2,0,24
Texas,Nighthawks,2nd,25,2,1099,62,3,1,31
Florida,Nighthawks,2nd,616,2,1400,26,3,1,2
Maine,Dragoons,1st,43,4,1592,73,2,0,3
Iowa,Dragoons,1st,234,7,1006,37,1,1,4
Alaska,Dragoons,2nd,523,8,987,949,2,0,24
Washington,Dragoons,2nd,62,3,849,48,3,1,31
Oregon,Scouts,1st,62,4,973,48,2,0,2


In [99]:
donor = demo_df.iloc[demo_df.dropna().distance.idxmin()]
donor

feature a       2.743726
feature b       4.506330
feature c       2.620240
feature d       1.362915
uncorrelated    0.011719
distance        1.035106
Name: 8, dtype: float64

In [104]:
demo_df.style.applymap(red, subset=pd.IndexSlice[8, 'feature a']).applymap(green, subset=pd.IndexSlice[8, 'distance'])

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,nan,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085


In [161]:
def spotlight_donors(df, donors, missing=None):
    s = df.style
    def green(x):
        return 'background-color: green; color: white'
    def darkgreen(x):
        return 'background-color: darkgreen; color: white'
    def lightgreen(x):
        return 'background-color: lightgreen'
    def yellow(x):
        return 'background-color: yellow'
    for each in donors.index:
        s=s.applymap(green, subset=pd.IndexSlice[each, 'distance']).applymap(lightgreen, subset=pd.IndexSlice[each,'feature a'])
        if 'regression' in df.columns:
            s=s.applymap(darkgreen, subset=pd.IndexSlice[each, 'regression'])
            if missing:
                s=s.applymap(yellow, subset=pd.IndexSlice[missing, 'regression'])
    return s

In [114]:
spotlight_donors(demo_df, [2,3])

,feature a,feature b,feature c,feature d,uncorrelated,distance
0,1.517509,4.229258,2.052726,0.153278,0.014975,2.140696
1,2.536323,4.295391,2.104137,1.348000,0.998701,1.679696
2,4.043034,5.872276,3.559629,3.274061,0.403823,1.804759
3,0.082752,3.761743,-0.440590,1.031832,0.281023,3.954464
4,0.196684,3.793343,1.016462,-0.667764,0.165431,3.477212
5,2.560068,4.446726,2.420763,0.973363,0.166179,1.312528
6,4.027199,5.079975,4.582185,0.876607,0.420479,1.651431
7,nan,5.339294,3.138633,1.611132,0.229141,0.000000
8,2.743726,4.506330,2.620240,1.362915,0.011719,1.035106
9,-0.180238,3.148906,0.280848,-0.741796,0.104471,4.303085
